Este documento contem o código do enunciado prático nº5, relativo a LSTM's, um caso particular das RNN's. 

Criação de Datasets para treino;teste;validação/Arquitetura RNN/Teste/Análise de resultados/Optimização de hiper parametros, arquitetura e afins/Conclusões finais

### Imports de bibliotecas & Setup

In [ ]:
import pandas as pd
import seaborn as sns
import io
import requests
import numpy as np
import tensorflow as tf

import os
import math
import joblib
import seaborn as sns
from tabulate import tabulate
from datetime import datetime

# fix random seed for reproducibility
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.core import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow import keras


from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
 
import matplotlib.pyplot as plt
import datetime as dt
from math import sqrt
from numpy import concatenate
 
#for replicability purposes
tf.random.set_seed(91195003)
np.random.seed(91190530)
 
#for an easy reset backend session state
tf.keras.backend.clear_session()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline


### Import do dataset e visualização inicial dos dados


In [ ]:
#Load dataset
def load_dataset(url,index):
    print("--"*25+" Loading Data "+"--"*25 )
    if url is 'Braga':
        url = 'https://raw.githubusercontent.com/A82516/CSC/main/Traffic_Incidents_Braga_Until_20191231.csv?token=AIX26P755Q55LRFKSMHCPYLAWACDG'
    if url is 'crypto':
        url ='https://drive.google.com/u/0/uc?id=1Yh-EYAXCHT2bkO433WbH5HKfXDKpUFQ-&export=download'
    data = requests.get(url).content
    df = pd.read_csv(io.StringIO(data.decode('utf-8')),index_col=index)
    
    return df

def distinct_df(df):
    object_columns = df.select_dtypes(include=["object"]).columns
    print("Contagem de valores únicos para as colunas categóricas do dataset:")
    for x in range(len(object_columns)):
        print("\n*---------- {} ----------*\n".format(object_columns[x]))
        print(df[object_columns[x]].value_counts()) 
        
def ResumeInfo(x):
    print("--"*25+" Resume of Dataset "+"--"*25 )
    print(tabulate(x.head(1), headers='keys', tablefmt='psql'))
    print("--"*25+"        Shape      "+"--"*25 )
    print(x.shape)
    print("--"*25+"        Info       "+"--"*25 )
    print(x.info())
    print("--"*25+"     Describe      "+"--"*25 )
    print(x.describe())
    print("--"*25+"    Null values    "+"--"*25 )
    nullseries = df_raw.isnull().sum()  
    print(nullseries[nullseries > 0])

    print(sns.heatmap(x.isnull(), yticklabels=False,cbar=False,cmap='viridis') )


In [ ]:
df_raw = load_dataset('Braga',None)
ResumeInfo(df_raw)

In [ ]:
#Análise da correlação entre as métricas do dataset
sns.heatmap(df_raw.corr(), cmap = 'coolwarm', vmin = -1, vmax = 1, linewidths=.5)
corr_l_long = df_raw['length_in_meters'].corr(df_raw['longitude'])
corr_lat_long = df_raw['latitude'].corr(df_raw['longitude'])
print("Correlacao l_long = " + str(corr_l_long) + ", Correlacao lat_long = " + str(corr_lat_long) + "\n" )
print(df_raw.corr())

In [ ]:
sns.heatmap(df_raw.corr())

In [ ]:
distinct_df(df_raw)

In [ ]:
num_attributes = df_raw.select_dtypes(include=['int64','float64'])

num_attributes.hist(figsize=(6,3))
plt.rc('xtick', labelsize=10)     
plt.rc('ytick', labelsize=10)

plt.show()

In [ ]:
sns.boxplot(x= df_raw["delay_in_seconds"], linewidth=2)

In [ ]:
sns.countplot(x = df_raw['incident_category_desc'])

In [ ]:
sns.countplot(x = df_raw['magnitude_of_delay_desc'])

### Tratamento de dados



Agora que observamos o tipo de dados que temos no dataset disponibilizado, podemos então tirar algumas conclusões sobre que tratamento de dados irá ser aplicado ao nosso dataset. 

Notas importantes: 
> A única cidade presente no dataset é Braga, tendo este 83347 entradas.

> Existem duas colunas com muitos valores *null*, nomeadamente "*cause_of_incidents*" e "*affected_roads*" (51% Null values) , tendo de ser tratados na próxima fase.
```
affected_roads             42366
cause_of_incident          82921
```

> Duas colunas apresentam uma forte correlação entre elas, sendo estas a coluna "*longitude*" com "*latitude*" e "*longitude*" com "*length_in_meters*". 
```
Correlacao l_long = -0.7555864468352008, Correlacao lat_long = -0.5883661254866525
```
> A coluna "*indident_date*" está em formato *datatime* e para que podessemos trabalhar melhor com esta coluna, criamos uma coluna detalhada para o ano, mês, dia, hora, minuto e segundo em que o acidente aconteceu.

> Foi importante observar os diferentes valores das colunas categóricas do nosso dataset para perceber a disparidade destes e que tipo de informação nos poderiam dar sobre o incidente. É de notar que existem duas colunas, "*affected_roads*" e "*to_road*", com 621 e 578 valores únicos, respetivamente, tendo uma disparidade muito grande de registos entre cada um. Podemos fazer algum tratamento de dados neste tipo de colunas para tentar diminuir esta discrepância. Nas restantes colunas categóricas também existe este desbalanceamento entre valores que deve ser analisado e ver se faz sentido ou não, no contexto do problema, ou se apenas não temos dados registados para algumas ocasiões específicas. 

> Devemos também analisar os gráficos que fizemos para extrair informação dos dados fornecidos, tirando algumas conclusões sobre que dados de incidentes rodoviários estão presentes no nosso dataset.

> Em termos de fases temporais em que ocorrem mais incidentes, podemos destacar os meses de novembro, dezembro e janeiro como tendo maior ocorrência de acidentes no nosso dataset. As horas em que ocorrem maiores acidentes é entre as 16-18 horas. Também podemos ver que ocorrem mais incidentes entre dia 10 e 22 de cada mês.  

In [ ]:
#Criação de uma coluna para cada espaço temporal presente na coluna incident_date

df_raw['incident_date'] = pd.to_datetime(df_raw['incident_date'])
df_raw['hour']          = df_raw['incident_date'].dt.hour
df_raw['Week']          = df_raw.apply(lambda row: row.incident_date.isocalendar()[1], axis=1)
df_raw['DayOfWeek']     = df_raw.apply(lambda row: row.incident_date.isoweekday(), axis=1)
df_raw.head(1)

#Drop "city_name" column as the values are always the same "Braga"
#Drop "cause of incident" column as the values are all null 

df_raw.drop(columns=['cause_of_incident','incident_date','affected_roads','city_name','from_road','to_road'], inplace=True)
df_raw = df_raw[df_raw.incident_category_desc != 'Unknown Incident']
df_raw = df_raw[df_raw.description  != 'incident']

df_raw.head(1)

In [ ]:
#magnitude_of_delay_desc

num_attributes = df_raw.select_dtypes(include=['int64','float64'])

df_raw["magnitude_of_delay_desc"].replace({'Unknown Delay': 'Undefined'}, inplace=True)

df_raw.head(1)

In [ ]:
df_raw = pd.get_dummies(df_raw, columns=['magnitude_of_delay_desc'], prefix='magnitude')
df_raw = pd.get_dummies(df_raw, columns=['incident_category_desc'], prefix='category')
df_raw = pd.get_dummies(df_raw, columns=['description'], prefix='desc')
df_raw.head(1)

In [ ]:
# split = df_raw['from_road'].str.extract(r'\(([^)]+)\)')
# print(split)

object_columns = df_raw.select_dtypes(include=["object"]).columns

print("Contagem de valores únicos para as colunas categóricas do dataset:")

for x in range(len(object_columns)):
    print("\n*---------- {} ----------*\n".format(object_columns[x]))
    print(df_raw[object_columns[x]].value_counts()) 
    print(df_raw[object_columns[x]].count()) 

Tratar das colunas com grande correlação entre elas, na minha opiniao faz sentido o lenght in meters ter correlaçao alta com a longitude e a latitude com longitude também
Tratar do desbalanceamento do dataset, mais incidente nas colunas to road e affected roads, temos de ver se faz sentido haver este desbalanceamento ou se devemos aplicar alguma técnica de undersampling
Em relação aos gráficos, é ver o range de valores que cada coluna tem e quais os valores que se destacam. 
Em relação as colunas com dados categóricos, podiamos agrupar alguns deles e posteriormente aplicar uma técnica de one-hot-encoding, criando uma coluna boleana para cada valor único das colunas categóricas
Vamos tentar prever o número de incidentes futuros, num certo período de tempo, forecasting basicamente. 
Depois de todo o tratamento de dados estar concluido, devemos aplicar a ambos os datasets que criamos, teste e treino, o mesmo tratamento. De seguida, começar a arquitetura da rede. 
De notar que a função split_data ainda não foi usada, por isso podemos apenas aplicar na próxima fase "criaçao de datasets"

### Criação dos datasets de Treino, Teste e Validação

In [ ]:
#Plot time series data
def plot_incidentes(data):
    plt.figure(figsize=(8,6))
    plt.plot(range(len(data)), data)
    plt.title('Incidentes ocorridos em Braga por dia')
    plt.ylabel('Incidentes')
    plt.xlabel('Dias')
    plt.show()

#split data into training and validation sets
def split_data(training, perc=10):
    train_idx = np.arange(0, int(len(training)*(100-perc)/100))
    val_idx = np.arange(int(len(training)*(100-perc)/100+1), len(training))
    return train_idx, val_idx

In [ ]:
##Normalização de colunas inteiras 

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))

for x in num_attributes:
  df_raw[x] = scaler.fit_transform(np.array(df_raw[x]).reshape(-1,1))

df_raw.head(5)

In [ ]:
col_names = ['length_in_meters','latitude', 'longitude', 'hour', 
             'Week', 'DayOfWeek', 'magnitude_Major',
             'magnitude_Minor', 'magnitude_Moderate', 'magnitude_Undefined',
             'category_Jam', 'category_Road Closed', 'category_Road Works',
             'desc_bridge closed', 'desc_closed', 'desc_queuing traffic',
             'desc_roadworks', 'desc_slow traffic', 'desc_stationary traffic',
             'desc_tunnel closed', 'delay_in_seconds']

df_raw = df_raw.reindex(columns=col_names)

df_raw.columns

In [ ]:
df_raw.dtypes

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
dataset = df_raw.copy()

values = dataset.values

dataAll = series_to_supervised(df_raw, 1, 1)
print(dataAll.head())

# split into train and test sets
values = dataAll.values
n_train_sample = round(len(dataset.copy()) * 0.8)
train = values[:n_train_sample, :]
test = values[n_train_sample:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

### Definição dos modelos


In [ ]:
def build_model(train_X=train_X, dropout_rate=0.6):
  # design network
  model = Sequential()
  model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
  model.add(Dropout(dropout_rate))
  model.add(LSTM(256, return_sequences=True, dropout=dropout_rate))
  model.add(LSTM(512, dropout=dropout_rate))
  model.add(Dense(50, activation='tanh'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1, activation='linear'))
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss=rmse, optimizer=opt, metrics = ['mae', rmse])
  return model
  
#Building the model
def rmse(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

# fit network
model = build_model()

train_X = np.asarray(train_X).astype('float32')
train_y = np.asarray(train_y).astype('float32')

history = model.fit(train_X, train_y, epochs=30, batch_size=32, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
dataset = df_raw.copy()

# values = dataset.values

# # split into train and test sets
n_train_sample = round(len(dataset.copy()) * 0.8)
train = values[:n_train_sample, :]
test = values[n_train_sample:, :]
# # split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
#LG


# logreg = LogisticRegression(
#                             solver = "newton-cg", 
#                             tol = 1e-4, 
#                             penalty = "l2", 
#                             C = 0.25, 
#                             max_iter = 100, 
#                             multi_class = "auto",
#                             dual = False, 
#                             n_jobs = -1 
#                             ).fit(train_X, train_y)


# logreg.score(train_X, train_y)

# predLG = logreg.predict(test_X)

# print(confusion_matrix(test_y, predLG))

# print('\n')

# print(classification_report(test_y, predLG))

In [ ]:
# Random forest 



# rfc = RandomForestClassifier(
#                             criterion = "gini",
#                             max_depth = 6,
#                             max_features = "auto",
#                             min_samples_leaf = 2,
#                             n_estimators = 500,
#                             verbose = 0
#                             ).fit(train_X, train_y)

# rfc.score(train_X, train_y)

# predRFC = rfc.predict(test_X)

# print(confusion_matrix(test_y, predRFC))

# print('\n')

# print(classification_report(test_y, predRFC))

# Crypto

In [ ]:
#df_a = pd.read_csv('/content/main.csv')
 
def TimeStampToTime(df,table):
    df[table] = df[table] // 1000
    df[table] = df[table].apply(dt.datetime.fromtimestamp)
    return df

def create_dataset(dataset, look_back=5):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# Dataset From https://www.kaggle.com/aakashverma8900/bitcoin-price-usd
df_crypto = load_dataset('crypto','Open Time')

ResumeInfo(df_crypto)
#Apenas ha valores numericos por isso é inutil fazer:
# distinct_df(df_a)

df_crypto.index = pd.to_datetime(df_crypto.index, unit='ms')
df_crypto = df_crypto[~df_crypto.index.duplicated(keep='first')]
df_crypto = df_crypto.resample('1T').pad()

#df_crypto = TimeStampToTime(df_crypto,'Open Time')
#df_crypto = TimeStampToTime(df_crypto,'Close Time')


In [ ]:
plt.figure(figsize=(40,7))
plt.plot(df_crypto.index[-100000:], df_crypto.Close[-100000:])
plt.show()

# plots fixes mas desnecesarios
# plt.figure(figsize = (20, 25))
# plotnumber = 1
# time=['Open Time','Close Time']
# for column in bitcoin.drop(columns=time):
#     if plotnumber <= len(bitcoin.columns):
#         ax = plt.subplot(3, 3, plotnumber)
#         sns.distplot(bitcoin[column])
#         plt.xlabel(column, fontsize = 15)
#     plotnumber += 1
# plt.show()

In [ ]:
plt.figure(figsize=(25,5.5))
plt.subplot(1,3,1)
corr=df_crypto.corr()
print(corr)
sns.heatmap(corr,vmin=0,vmax=1)
plt.subplot(1,3,2)
sns.heatmap(df_crypto[['Open','High','Low','Close']].corr(),vmin=0,vmax=1)
plt.subplot(1,3,3)
corr=df_crypto.drop(columns=['Open','High','Low','Close']).corr()
sns.heatmap(corr,vmin=0,vmax=1)
plt.tight_layout()

In [ ]:
dataset = df_crypto['Close'][-100000:].values
dataset = dataset.astype('float32')
dataset = dataset.reshape(-1, 1)
dataset

In [ ]:
# create and fit the LSTM network
start = time.time()
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
for i in range(50):
    model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()
    print('done ', i)
end = time.time()
print(end - start)

In [ ]:
# Normalização do Dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# Separação dos dados em Traino e teste
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))
# reshape into X=t and Y=t+1
look_back = 6
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
# make predictions
trainPredict = model.predict(trainX,  batch_size=batch_size)
testPredict = model.predict(testX,  batch_size=batch_size)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.figure(figsize=(15,10))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()